# Sensor Data Fusion 
Lecturer: Prof. Baum \
Tutor: Kolja Thormann\
Semester: Winter 21/22

## Homework 6

Consider again the setup of the Silly Estimator exercise with $x \sim \mathcal{N}(\mu\_1, \Sigma\_1)$ and $y = x + e$ with $e \sim \mathcal{N}(\mu\_2, \Sigma\_2)$.


---
The following tasks will have missing sections marked that you should fill out. 

Missing code parts are marked by
```
# ... code code code
=== YOUR CODE HERE ===

=== END OF YOUR CODE ===
# ... code code code
```
If you are asked to implement a function, make sure to check what variable will be returned by the function and to fill it accordingly. Do not change code outside of the indicated sections.

Furthermore, some questions require theoretical answers instead of python code.

Such questions will have a field marked like this: 

=== YOUR ANSWER HERE === 

---
### a)

Write a function which calculates a possible value for $x$ based on its distribution and then generates a measurement from the value using the measurement noise.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def draw_sample(mean_x, var_x, mean_e, var_e, sample_count):
    """
    Function that draws a sample measurement.
    The parameters for this function consist of the parameters for the normal distribution of x and the normal distribution 
    of the additive noise.
    
    The function should return a single measurement ("y") and the true drawn value from the distribution ("x"). 
    :param mean_x: Mean of the gaussian distribution for x
    :param var_x: Variance of the gaussian distribution for x
    :param mean_e: Mean of the gaussian distriubtion for e
    :param var_e: Variance of the gaussian distribution for e:
    :return: (y,x) - a single measurement y with its true (original) value x
    """
    
    # === YOUR CODE HERE ===

    y = []
    x = []

    for curr_run in range(sample_count):
        curr_x = np.random.multivariate_normal(mean_x, var_x)
        curr_e = np.random.multivariate_normal(mean_e, var_e)

        x.append(curr_x)
        y.append(curr_x + curr_e)

    # === END OF YOUR CODE ===
    
    return np.array(y), np.array(x)

visualize a given list of points

In [ ]:
def visualize_points(points):
    x_vals = points[:,0]
    y_vals = points[:,1]

    plt.scatter(x_vals, y_vals)
    plt.show()

---
### b)

Consider the natural estimator $\theta_n(y) = y$. Calculate the empirical mean square error of 1000 runs using this estimator and compare it with the analystic solution.

In [ ]:
def theta_n(y):
    """
    Function that represents the natural estimator.
    
    This function should return a single value, which represents the result of the estimator.
    :param y: The measurement y
    :return: The estimate corresponding to the measurement given as input
    """
    # === YOUR CODE HERE ===

    result = y
    
    # === END OF YOUR CODE ===
    
    return result

In [ ]:
def empirical_mean_square_error(estimator, samples, n_runs=1000):
    """
    Function that calculates the empirical MSE for a given estimator and a set number of runs, using the draw_sample function
    defined above.
    
    The function should return a single value, which is the MSE over all runs for the given estimator.
    :param estimator: A function that, given a single measurement, returns an estimated true value
    :param n_runs: How many runs to take the MSE over
    :return: MSE, the mean squared error over all runs
    """
    # === YOUR CODE HERE ===

    for curr_run in n_runs:
        x = draw_sample()
        y = estimator()
    
    # === END OF YOUR CODE ===
    
    return MSE

In [ ]:

samples = draw_sample()
# Calculate the MSE for the natural estimator
empirical_MSE_natural = empirical_mean_square_error(estimator=theta_n, n_runs=1000)

print("Empirical MSE for the natural estimator: ")
print(empirical_MSE_natural)

Compare the result of the empirical testing with the analytic solution:

=== YOUR ANSWER HERE ===


---
__c)__ Repeat the process with the sillty estimator $\theta_s(y) = 0$

In [ ]:
def theta_s(y):
    """
    Function that represents the silly estimator.
    
    This function should return a single value, which represents the result of the estimator.
    :param y: The measurement y
    :return: The estimate corresponding to the measurement given as input
    """
    # === YOUR CODE HERE ===
    
    
    # === END OF YOUR CODE ===
    
    return result

In [ ]:
# Calculate the MSE for the silly estimator using the same function as before
empirical_MSE_silly = empirical_mean_square_error(estimator=theta_s, n_runs=1000)

print("Empirical MSE for the silly estimator: ")
print(empirical_MSE_silly)

Compare the result of the empirical testing with the analytic solution, this time for $\theta_s$:

=== YOUR ANSWER HERE ===
